In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout, TimeDistributed
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-03-22 00:04:01.458961: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 00:04:01.465953: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 00:04:01.484603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742591041.518711 1358780 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742591041.529013 1358780 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 00:04:01.566560: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=False))

    model.add(Dropout(0.2))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [ ]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = [1, 2]
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,...,Raw_2,Delta_2,Theta_2,Alpha1_2,Alpha2_2,Beta1_2,Beta2_2,Gamma1_2,Gamma2_2,predefinedlabel
0,0,0.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,...,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-03-22 00:04:06.082241: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 144, 27)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc (LTC)                       │ (None, 144, 10)        │         1,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 144, 10)        │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_2 (LTC)                     │ (None, 10)             │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,397 (13.27 KB)

 Trainable params: 3,397 (13.27 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5471 - auc: 0.6567 - loss: 0.6901 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6948
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 844ms/step - accuracy: 0.4217 - auc: 0.5439 - loss: 0.7040 - val_accuracy: 0.4333 - val_auc: 0.5000 - val_loss: 0.6934
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 767ms/step - accuracy: 0.5662 - auc: 0.5352 - loss: 0.6934 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 845ms/step - accuracy: 0.4370 - auc: 0.4516 - loss: 0.7076 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 857ms/step - accuracy: 0.5193 - auc: 0.4788 - loss: 0.7040 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 962ms/step - accuracy: 0.4507 - auc: 0.3827 - loss: 0.7130 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 727ms/st

1it [23:43, 1423.22s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5668 - auc: 0.6022 - loss: 0.6808 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6974
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 813ms/step - accuracy: 0.5753 - auc: 0.5490 - loss: 0.6812 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6893
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 773ms/step - accuracy: 0.5572 - auc: 0.5873 - loss: 0.6755 - val_accuracy: 0.5000 - val_auc: 0.5333 - val_loss: 0.6925
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 935ms/step - accuracy: 0.4067 - auc: 0.3650 - loss: 0.7093 - val_accuracy: 0.5667 - val_auc: 0.6000 - val_loss: 0.6915
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 784ms/step - accuracy: 0.6326 - auc: 0.6064 - loss: 0.6847 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6856
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 800ms/step - accuracy: 0.4883 - auc: 0.5179 - loss: 0.6855 - val_accuracy: 0.6000 - val_auc: 0.7000 - val_loss: 0.6835
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 978ms

2it [48:15, 1452.19s/it]

...

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.4786 - auc: 0.4708 - loss: 0.7127 - val_accuracy: 0.5000 - val_auc: 0.6667 - val_loss: 0.6934
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 856ms/step - accuracy: 0.3773 - auc: 0.3086 - loss: 0.7332 - val_accuracy: 0.5000 - val_auc: 0.4667 - val_loss: 0.6934
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 768ms/step - accuracy: 0.5580 - auc: 0.5686 - loss: 0.6847 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6931
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 825ms/step - accuracy: 0.5047 - auc: 0.4844 - loss: 0.6968 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6921
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 865ms/step - accuracy: 0.4947 - auc: 0.4748 - loss: 0.6992 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6919
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 868ms/step - accuracy: 0.5820 - auc: 0.5534 - loss: 0.6909 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6907
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 862ms/s

4it [1:38:20, 1488.15s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5694 - auc: 0.4602 - loss: 0.6978 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 828ms/step - accuracy: 0.5559 - auc: 0.5545 - loss: 0.6794 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 882ms/step - accuracy: 0.6277 - auc: 0.5070 - loss: 0.6928 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 972ms/step - accuracy: 0.4232 - auc: 0.5624 - loss: 0.6926 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 816ms/step - accuracy: 0.5095 - auc: 0.5206 - loss: 0.6953 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 824ms/step - accuracy: 0.5300 - auc: 0.4833 - loss: 0.6942 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 824ms/

5it [2:03:32, 1482.56s/it]

CPU times: user 3h 30min 13s, sys: 59min 34s, total: 4h 29min 47s
Wall time: 2h 3min 32s


In [ ]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

Epoch 1: TRAIN Accuracy = 0.506 Loss = 0.701 AUC = 0.536
Epoch 1: VAL Accuracy = 0.5 Loss = 0.696 AUC = 0.607
Epoch 2: TRAIN Accuracy = 0.46 Loss = 0.704 AUC = 0.476
Epoch 2: VAL Accuracy = 0.487 Loss = 0.692 AUC = 0.547
Epoch 3: TRAIN Accuracy = 0.54 Loss = 0.691 AUC = 0.546
Epoch 3: VAL Accuracy = 0.5 Loss = 0.693 AUC = 0.553
Epoch 4: TRAIN Accuracy = 0.477 Loss = 0.696 AUC = 0.477
Epoch 4: VAL Accuracy = 0.513 Loss = 0.693 AUC = 0.573
Epoch 5: TRAIN Accuracy = 0.531 Loss = 0.696 AUC = 0.512
Epoch 5: VAL Accuracy = 0.52 Loss = 0.691 AUC = 0.573
Epoch 6: TRAIN Accuracy = 0.48 Loss = 0.698 AUC = 0.472
Epoch 6: VAL Accuracy = 0.52 Loss = 0.691 AUC = 0.593
Epoch 7: TRAIN Accuracy = 0.526 Loss = 0.693 AUC = 0.534
Epoch 7: VAL Accuracy = 0.52 Loss = 0.69 AUC = 0.573
Epoch 8: TRAIN Accuracy = 0.531 Loss = 0.692 AUC = 0.524
Epoch 8: VAL Accuracy = 0.52 Loss = 0.69 AUC = 0.573
Epoch 9: TRAIN Accuracy = 0.554 Loss = 0.688 AUC = 0.547
Epoch 9: VAL Accuracy = 0.52 Loss = 0.689 AUC = 0.593
Epoch 